In [15]:
import os
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
x_train = []
y_train = []

len_images = os.listdir("/content/drive/MyDrive/Colab Notebooks/lab4/training/len/")
shock_images  = os.listdir("/content/drive/MyDrive/Colab Notebooks/lab4/training/shock/")


for i in range(len(len_images)):
    if len_images[i].startswith("."):
        continue

    image = Image.open("/content/drive/MyDrive/Colab Notebooks/lab4/training/len/" + len_images[i])
    converted_size = image.resize((28,28))
    converted_color = converted_size.convert('L')
    pixels_line = np.array((np.asarray(converted_color) / 255).astype(np.float32))
    x_train.append(pixels_line)
    y_train.append([1,0])

for j in range(len(shock_images)):
    if shock_images[j].startswith("."):
        continue

    image = Image.open("/content/drive/MyDrive/Colab Notebooks/lab4/training/shock/" + shock_images[j])
    converted_size = image.resize((28,28))
    converted_color = converted_size.convert('L')
    pixels_line = (np.asarray(converted_color) / 255).astype(np.float32)
    x_train.append(pixels_line)
    y_train.append([0,1])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.expand_dims(x_train, -1)

In [18]:
x_test = []
y_test = []

len_images = os.listdir("/content/drive/MyDrive/Colab Notebooks/lab4/test/len/")
shock_images  = os.listdir("/content/drive/MyDrive/Colab Notebooks/lab4/test/shock/")

for i in range(len(len_images)):
    if len_images[i].startswith("."):
        continue

    image = Image.open("/content/drive/MyDrive/Colab Notebooks/lab4/test/len/" + len_images[i])
    converted_size = image.resize((28,28))
    converted_color = converted_size.convert('L')
    pixels_line = np.array((np.asarray(converted_color) / 255).astype(np.float32))
    x_test.append(pixels_line)
    y_test.append([1,0])

for j in range(len(shock_images)):
    if shock_images[j].startswith("."):
        continue

    image = Image.open("/content/drive/MyDrive/Colab Notebooks/lab4/test/shock/" + shock_images[j])
    converted_size = image.resize((28,28))
    converted_color = converted_size.convert('L')
    pixels_line = (np.asarray(converted_color) / 255).astype(np.float32)
    x_test.append(pixels_line)
    y_test.append([0,1])

x_test = np.array(x_test)
y_test = np.array(y_test)

x_test = np.expand_dims(x_test, -1)

1 Модель

In [19]:
model_1 = keras.Sequential([
    Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")
])

model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 3136)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [20]:
model_1.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [21]:
y_train.shape

(28, 2)

In [22]:
h = model_1.fit(x_train, y_train, batch_size=32, epochs=25, validation_split=0.2)

Epoch 1/25
1/1 [==============================] - 1s 705ms/step - loss: 0.6891 - accuracy: 0.5455 - val_loss: 0.7828 - val_accuracy: 0.0000e+00
Epoch 2/25
1/1 [==============================] - 0s 63ms/step - loss: 0.6423 - accuracy: 0.5909 - val_loss: 0.6186 - val_accuracy: 1.0000
Epoch 3/25
1/1 [==============================] - 0s 74ms/step - loss: 0.5977 - accuracy: 1.0000 - val_loss: 0.6952 - val_accuracy: 0.3333
Epoch 4/25
1/1 [==============================] - 0s 60ms/step - loss: 0.5500 - accuracy: 0.8636 - val_loss: 0.6579 - val_accuracy: 0.5000
Epoch 5/25
1/1 [==============================] - 0s 63ms/step - loss: 0.4974 - accuracy: 0.8636 - val_loss: 0.4934 - val_accuracy: 1.0000
Epoch 6/25
1/1 [==============================] - 0s 71ms/step - loss: 0.4446 - accuracy: 1.0000 - val_loss: 0.5623 - val_accuracy: 0.6667
Epoch 7/25
1/1 [==============================] - 0s 65ms/step - loss: 0.3921 - accuracy: 0.9545 - val_loss: 0.4189 - val_accuracy: 1.0000
Epoch 8/25
1/1 [======

In [23]:
model_1.evaluate(x_test, y_test)

1/1 [==============================] - 0s 24ms/step - loss: 0.1930 - accuracy: 0.7500


[0.193044975399971, 0.75]

2 Модель

In [24]:
model_2 = keras.Sequential([
    Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(128, (3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(128, (3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(128, (3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")
])

model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 7, 7, 128)         147584    
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 3, 3, 128)       

In [25]:
model_2.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [26]:
y_train.shape

(28, 2)

In [27]:
h = model_2.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20
1/1 [==============================] - 1s 894ms/step - loss: 0.6928 - accuracy: 0.5455 - val_loss: 0.7678 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 147ms/step - loss: 0.6849 - accuracy: 0.5455 - val_loss: 0.8472 - val_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 145ms/step - loss: 0.6772 - accuracy: 0.5455 - val_loss: 0.8443 - val_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 141ms/step - loss: 0.6647 - accuracy: 0.5455 - val_loss: 0.8003 - val_accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 0s 140ms/step - loss: 0.6450 - accuracy: 0.5455 - val_loss: 0.8046 - val_accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 0s 141ms/step - loss: 0.6182 - accuracy: 0.5909 - val_loss: 0.8086 - val_accuracy: 0.1667
Epoch 7/20
1/1 [==============================] - 0s 138ms/step - loss: 0.5811 - accuracy: 0.6364 - val_loss: 0.6367 - val_accuracy: 0.6667


In [28]:
model_2.evaluate(x_test, y_test)

1/1 [==============================] - 0s 25ms/step - loss: 0.1412 - accuracy: 1.0000


[0.14118102192878723, 1.0]